In [1]:
import numpy as np
import pandas
from tqdm import tqdm
from ase.build import bulk

In [2]:
from pympipool import Pool

In [3]:
def body_order(n=32, b=5):
    if b == 2:
        return [[i ,n-i] for i in range(n+1)]
    else:
        lst = []
        for i in range(n+1):
            for j in body_order(n=n-i, b=b-1):
                lst.append([i] + j)
        return lst

In [4]:
def get_elastic_constants(input_para):
    i, conc_lst, number_atoms, element_lst, potential, structure = input_para

    # import 
    import numpy as np
    import pyiron_contrib
    from pyiron_atomistics import Project, ase_to_pyiron

    # create project
    project = Project('workflow')

    # Generate structure
    if sum(np.array(conc_lst)==0) != 4:
        mole_fraction = {
            el: conc/number_atoms
            for el, conc in zip(element_lst, conc_lst)
            if conc > 0
        }
        job = project.create_job(project.job_type.SQSJobWithoutOutput, "sqs_" + str(i))
        job._interactive_disable_log_file = True
        job.structure = ase_to_pyiron(structure)
        job.input['mole_fractions'] = mole_fraction
        job.input['iterations'] = 1e6
        job.server.cores = 1
        job.run()
        structure_next = job._lst_of_struct[-1]
    else:
        structure_next = ase_to_pyiron(structure).copy()
        structure_next.symbols[:] = np.array(element_lst)[np.array(conc_lst)!=0][0]

    # Minimize strucute
    lmp_mini1 = project.create_job(project.job_type.LammpsInteractiveWithoutOutput, "lmp_mini_" + str(i), delete_existing_job=True)
    lmp_mini1.structure = structure_next
    lmp_mini1.potential = potential
    lmp_mini1.calc_minimize(pressure=0.0)
    lmp_mini1.server.run_mode.interactive = True
    lmp_mini1.interactive_mpi_communicator = MPI.COMM_SELF
    lmp_mini1._interactive_disable_log_file = True  # disable lammps.log
    lmp_mini1.run()
    lmp_mini1.interactive_close()

    # Elastic constants
    lmp_elastic = project.create_job(project.job_type.LammpsInteractiveWithoutOutput, "lmp_elastic_" + str(i), delete_existing_job=True)
    lmp_elastic.structure = lmp_mini1.get_structure()
    lmp_elastic.potential = potential
    lmp_elastic.interactive_enforce_structure_reset = True
    lmp_elastic.interactive_mpi_communicator = MPI.COMM_SELF
    lmp_elastic.server.run_mode.interactive = True
    lmp_elastic._interactive_disable_log_file = True  # disable lammps.log
    elastic = lmp_elastic.create_job(project.job_type.ElasticMatrixJobWithoutFiles, "elastic_" + str(i), delete_existing_job=True)
    elastic._interactive_disable_log_file = True  # disable lammps.log
    elastic.run()

    elastic_constants_lst = [
        elastic._data["C"][0][0],
        elastic._data["C"][0][1],
        elastic._data["C"][0][2],
        elastic._data["C"][0][3],
        elastic._data["C"][0][4],
        elastic._data["C"][0][5],
        elastic._data["C"][1][1],
        elastic._data["C"][1][2],
        elastic._data["C"][1][3],
        elastic._data["C"][1][4],
        elastic._data["C"][1][5],
        elastic._data["C"][2][2],
        elastic._data["C"][2][3],
        elastic._data["C"][2][4],
        elastic._data["C"][2][5],
        elastic._data["C"][3][3],
        elastic._data["C"][3][4],
        elastic._data["C"][3][5],
        elastic._data["C"][4][4],
        elastic._data["C"][4][5],
        elastic._data["C"][5][5],
    ]

    if "Fe" in elastic.ref_job.structure.get_species_symbols():
        conc_Fe = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Fe")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Fe = 0.0
    if "Ni" in elastic.ref_job.structure.get_species_symbols():
        conc_Ni = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Ni")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Ni = 0.0
    if "Cr" in elastic.ref_job.structure.get_species_symbols():
        conc_Cr = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Cr")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Cr = 0.0
    if "Co" in elastic.ref_job.structure.get_species_symbols():
        conc_Co = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Co")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Co = 0.0
    if "Cu" in elastic.ref_job.structure.get_species_symbols():
        conc_Cu = sum(elastic.ref_job.structure.indices==elastic.ref_job.structure.get_species_symbols().tolist().index("Cu")) / len(elastic.ref_job.structure.indices)
    else:
        conc_Cu = 0.0

    conc_lst = [conc_Fe, conc_Ni, conc_Cr, conc_Co, conc_Cu]
    
    return elastic_constants_lst + conc_lst

In [5]:
structure = bulk("Al", cubic=True).repeat([2,2,2])

In [6]:
element_lst = ["Fe", "Ni", "Cr", "Co", "Cu"]
number_atoms = len(structure)
number_species = len(element_lst)
potential = '2021--Deluigi-O-R--Fe-Ni-Cr-Co-Cu--LAMMPS--ipr1'

In [7]:
lst_32 = body_order(n=number_atoms, b=number_species)[:120]
input_para_lst = [
    [i, conc_lst, number_atoms, element_lst, potential, structure]
    for i, conc_lst in enumerate(lst_32)
]

In [8]:
with Pool(cores=12) as p:
    results = p.map(function=get_elastic_constants, lst=input_para_lst)

Configs:   0%|          | 0/120 [00:00<?, ?it/s]/home/janssen/miniforge3/lib/python3.9/site-packages/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/home/janssen/miniforge3/lib/python3.9/site-packages/pyiron_atomistics/lammps/interactive.py:121: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/home/janssen/miniforge3/lib/python3.9/site-packages/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/home/janssen/miniforge3/lib/python3.9/site-packages/pyiron_atomistics/lammps/interactive.py:491: UserWarning: Warning: setting upper trangular matrix might slow down the calculation
  warnings.warn(
/home/janssen/miniforge3/lib/python3.9/site-packages/pyiron_atomistics/lammps/interactive.py:121: UserWarning: Warning: setting upper trangular matrix m

In [9]:
c11_lst, c12_lst, c13_lst, c14_lst, c15_lst, c16_lst, c22_lst, c23_lst, c24_lst, c25_lst, c26_lst, c33_lst, c34_lst, c35_lst, c36_lst, c44_lst, c45_lst, c46_lst, c55_lst, c56_lst, c66_lst, conc_Fe_lst, conc_Ni_lst, conc_Cr_lst, conc_Co_lst, conc_Cu_lst = zip(*results)

In [10]:
df = pandas.DataFrame({
    "conc_Fe": conc_Fe_lst,
    "conc_Ni": conc_Ni_lst,
    "conc_Cr": conc_Cr_lst,
    "conc_Co": conc_Co_lst,
    "conc_Cu": conc_Cu_lst,
    "C11": c11_lst,
    "C12": c12_lst,
    "C13": c13_lst,
    "C14": c14_lst,
    "C15": c15_lst,
    "C16": c16_lst,
    "C22": c22_lst,
    "C23": c23_lst,
    "C24": c24_lst,
    "C25": c25_lst,
    "C26": c26_lst,
    "C33": c33_lst,
    "C34": c34_lst,
    "C35": c35_lst,
    "C36": c36_lst,
    "C44": c44_lst,
    "C45": c45_lst,
    "C46": c46_lst,
    "C55": c55_lst,
    "C56": c56_lst,
    "C66": c66_lst
})
df

,conc_Fe,conc_Ni,conc_Cr,conc_Co,conc_Cu,C11,C12,C13,C14,C15,...,C33,C34,C35,C36,C44,C45,C46,C55,C56,C66
0,0.0,0.0,0.00000,0.00000,1.00000,171.550807,116.387656,116.387656,0.000000,0.000000,...,171.550807,0.000000,0.000000,0.000000,76.350076,0.000000,0.000000,76.350076,0.000000,76.350076
1,0.0,0.0,0.00000,0.96875,0.03125,168.206377,117.342596,117.342596,0.000000,0.000000,...,168.206377,0.000000,0.000000,0.000000,78.135186,0.000000,0.000000,78.135186,0.000000,78.135186
2,0.0,0.0,0.00000,0.93750,0.06250,166.234934,118.955277,118.955277,0.000000,0.000000,...,170.331774,0.000000,0.000000,0.000000,79.999228,0.000000,0.000000,79.969330,0.000000,79.969330
3,0.0,0.0,0.00000,0.90625,0.09375,169.825220,119.995795,120.210260,0.000000,0.000000,...,171.454932,0.000000,0.000000,0.285802,81.911327,-0.047475,0.000000,81.880257,0.000000,81.911329
4,0.0,0.0,0.00000,0.87500,0.12500,174.056252,122.159376,121.950598,-0.041975,0.076585,...,175.446023,0.066813,0.073435,-0.062288,83.997544,0.024078,-0.063946,83.784199,-0.003049,83.893676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0.0,0.0,0.31250,0.09375,0.59375,223.107394,142.256339,142.070844,0.279557,-1.297722,...,226.588435,0.356755,-1.181956,0.011955,120.075521,0.003192,0.226314,119.926365,0.097574,121.086602
116,0.0,0.0,0.09375,0.62500,0.28125,224.344749,142.622149,142.648084,0.287226,0.362141,...,220.939720,0.215740,0.212558,-0.081292,122.101445,-0.133179,-0.039746,122.830218,0.051133,122.567832
117,0.0,0.0,0.25000,0.65625,0.09375,229.113037,148.194688,147.956846,-0.120379,-0.229024,...,229.895578,0.146122,-0.183579,0.141818,123.484716,-0.008214,0.026276,123.506539,-0.022794,123.840865
118,0.0,0.0,0.09375,0.21875,0.68750,226.734650,148.520268,150.734353,0.121997,-0.492772,...,234.749645,0.026191,-0.595170,0.151582,126.452171,0.110218,-0.031538,126.657576,0.205416,126.414791


In [11]:
df.to_hdf("elastic.h5", "elastic")